In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from casestudy.update import auto_update

In [4]:
auto_update()

NameError: name 'auto_update' is not defined